table of contents
1. 準備
2. データのインポート
3. EDA
4. サマリの完成

# 準備

In [1]:
import os, time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

cmap = plt.get_cmap('tab10')

# データのインポート

In [2]:
base_dir = os.path.join('..','..')
data_dir = os.path.join(base_dir, 'data', 'raw')

def read_dfs():
    players_df = pd.read_json(os.path.join(data_dir, 'players.json'))
    teams_df = pd.read_json(os.path.join(data_dir, 'teams.json'))

    # encode, decode
    players_df['shortName'] = players_df.shortName.apply(lambda x: x.encode('utf-8').decode('unicode-escape'))
    teams_df['name'] = teams_df.name.apply(lambda x: x.encode('utf-8').decode('unicode-escape'))

    return players_df, teams_df

players_df, teams_df = read_dfs()

competition_list = [infile.replace('matches_','').replace('.json','') for infile in os.listdir(os.path.join(data_dir, 'matches')) if infile.endswith('.json')]

print(competition_list)

['World_Cup', 'Italy', 'Germany', 'England', 'France', 'Spain', 'European_Championship']


In [3]:
selected_competition = 'European_Championship'

def read_events_df(selected_competition):
    events_df = pd.read_json(os.path.join(data_dir, 'events', f'events_{selected_competition}.json'))
    return events_df

def read_matches_df(selected_competition):
    matches_df = pd.read_json(os.path.join(data_dir, 'matches', f'matches_{selected_competition}.json'))
    matches_df['date_time'] = pd.to_datetime(matches_df.dateutc)

    # encode, decode
    matches_df['label'] = matches_df.label.apply(lambda x: x.encode('utf-8').decode('unicode-escape'))
    matches_df['venue'] = matches_df.venue.apply(lambda x: x.encode('utf-8').decode('unicode-escape'))

    # concate label, venue, date_time
    matches_df['name'] = matches_df[['label', 'venue', 'date_time']].apply(lambda xs: f'{xs[0]} @{xs[1]}, {xs[2].date()}', axis=1)

    return matches_df.sort_values('date_time')

matches_df = read_matches_df(selected_competition)

name_list = matches_df.name.tolist()
print(name_list)

['France - Romania, 2 - 1 @Stade de France, 2016-06-10', 'Albania - Switzerland, 0 - 1 @Stade Bollaert-Delelis, 2016-06-11', 'Wales - Slovakia, 2 - 1 @Stade Matmut-Atlantique, 2016-06-11', 'England - Russia, 1 - 1 @Stade Vélodrome, 2016-06-11', 'Turkey - Croatia, 0 - 1 @Parc des Princes, 2016-06-12', 'Poland - Northern Ireland, 1 - 0 @Allianz Riviera, 2016-06-12', 'Germany - Ukraine, 2 - 0 @Stade Pierre-Mauroy, 2016-06-12', 'Spain - Czech Republic, 1 - 0 @Stadium Municipal, 2016-06-13', 'Republic of Ireland - Sweden, 1 - 1 @Stade de France, 2016-06-13', 'Belgium - Italy, 0 - 2 @Groupama Stadium, 2016-06-13', 'Austria - Hungary, 0 - 2 @Stade Matmut-Atlantique, 2016-06-14', 'Portugal - Iceland, 1 - 1 @Stade Geoffroy-Guichard, 2016-06-14', 'Russia - Slovakia, 1 - 2 @Stade Pierre-Mauroy, 2016-06-15', 'Romania - Switzerland, 1 - 1 @Parc des Princes, 2016-06-15', 'France - Albania, 2 - 0 @Stade Vélodrome, 2016-06-15', 'England - Wales, 2 - 1 @Stade Bollaert-Delelis, 2016-06-16', 'Ukraine - N

In [4]:
selected_match = 'Germany - France, 0 - 2 @Stade Vélodrome, 2016-07-07'
selected_wyId = matches_df[matches_df.name == selected_match].wyId.values[0]

events_df = read_events_df(selected_competition)

df_tmp = events_df[(events_df.matchId == selected_wyId)]

# EDA

In [5]:
df_tmp.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
74554,8,Simple pass,[{'id': 1801}],25397,"[{'y': 33, 'x': 28}, {'y': 73, 'x': 29}]",1694439,Pass,4418,1H,1.908619,85,90356461
74555,8,Simple pass,[{'id': 1801}],7855,"[{'y': 73, 'x': 29}, {'y': 73, 'x': 32}]",1694439,Pass,4418,1H,5.177858,85,90356462
74556,8,Simple pass,[{'id': 1801}],7936,"[{'y': 73, 'x': 32}, {'y': 80, 'x': 26}]",1694439,Pass,4418,1H,7.405928,85,90356463
74557,8,High pass,[{'id': 1801}],7855,"[{'y': 80, 'x': 26}, {'y': 80, 'x': 64}]",1694439,Pass,4418,1H,11.910527,83,90356464
74558,1,Air duel,"[{'id': 703}, {'id': 1801}]",26010,"[{'y': 80, 'x': 64}, {'y': 60, 'x': 75}]",1694439,Duel,4418,1H,13.640313,10,90356465


In [6]:
df_tmp.isnull().sum()

eventId         0
subEventName    0
tags            0
playerId        0
positions       0
matchId         0
eventName       0
teamId          0
matchPeriod     0
eventSec        0
subEventId      0
id              0
dtype: int64

# サマリの完成
表示する情報は以下の通りである。
* スコア
* シュートの本数
* パス本数
* パス成功率
* クリア数
* ファール数
* デュエルの勝率  
(+alpha)
* パスの種類（layout.bar(subset=['cnt']）)

## スコア

In [55]:
score_s = df_tmp[df_tmp.eventName=='Shot'].groupby('teamId').tags.agg(lambda xs: np.sum([[101 in [tag_dict['id'] for tag_dict in tags]] for tags in xs])) 

penalty_s = df_tmp[df_tmp.subEventName=='Penalty'].groupby('teamId').tags.agg(lambda xs: np.sum([[101 in [tag_dict['id'] for tag_dict in tags]] for tags in xs])) 
for i in penalty_s.index:
    score_s[i] += penalty_s[i]

score_df = score_s.fillna('0').to_frame().T
score_df.index = ['score']

score_df

teamId,3148,4418
score,0,2


## シュートの本数

In [56]:
shot_s = df_tmp[df_tmp.eventName=='Shot'].groupby('teamId').size()

penalty_s = df_tmp[df_tmp.subEventName=='Penalty'].groupby('teamId').size()
for i in penalty_s.index:
    shot_s[i] += penalty_s[i]

shot_df = shot_s.fillna('0').to_frame().T
shot_df.index = ['number of shots']

shot_df

teamId,3148,4418
number of shots,15,12


## パス本数

In [57]:
pass_s = df_tmp[df_tmp.eventName=='Pass'].groupby('teamId').size()

pass_df = pass_s.fillna('0').to_frame().T
pass_df.index = ['number of passes']

pass_df

teamId,3148,4418
number of passes,644,270


## パス成功率

In [71]:
r_pass_s = df_tmp[df_tmp.eventName=='Pass'].groupby('teamId').tags.agg(lambda xs: np.sum([[1801 in [tag_dict['id'] for tag_dict in tags]] for tags in xs])) 

r_pass_df = (r_pass_s.to_frame().T / pass_df.values * 100).astype(int)
r_pass_df = r_pass_df.applymap(lambda x: f'{x}%')

r_pass_df.index = ['pass success rate']

r_pass_df

teamId,3148,4418
pass success rate,87%,81%


## クリア数

In [72]:
clear_s = df_tmp[df_tmp.subEventName=='Clearance'].groupby('teamId').size()

clear_df = clear_s.fillna('0').to_frame().T
clear_df.index = ['number of Clear']

clear_df

teamId,3148,4418
number of Clear,5,24


## ファール数

In [74]:
foul_s = df_tmp[df_tmp.subEventName=='Foul'].groupby('teamId').size()

foul_df = foul_s.fillna('0').to_frame().T
foul_df.index = ['number of Foul']

foul_df

teamId,3148,4418
number of Foul,7,12


## デュエルの勝率

In [94]:
r_duel_s = df_tmp[df_tmp.eventName=='Duel'].groupby('teamId').tags.agg(lambda xs:np.sum([np.sum([1801 in [tag_dict['id']] for tag_dict in tags]) for tags in xs])/len(xs))

r_duel_df = (r_duel_s.to_frame().T * 100).astype(int)
r_duel_df = r_duel_df.applymap(lambda x: f'{x}%')

r_duel_df.index = ['duel win rate']

r_duel_df

teamId,3148,4418
duel win rate,61%,61%


# 結合

In [105]:
def create_match_summary_df(df_tmp, teams_df):
    # スコア
    score_s = df_tmp[df_tmp.eventName=='Shot'].groupby('teamId').tags.agg(lambda xs: np.sum([[101 in [tag_dict['id'] for tag_dict in tags]] for tags in xs])) 

    penalty_s = df_tmp[df_tmp.subEventName=='Penalty'].groupby('teamId').tags.agg(lambda xs: np.sum([[101 in [tag_dict['id'] for tag_dict in tags]] for tags in xs])) 
    for i in penalty_s.index:
        score_s[i] += penalty_s[i]

    score_df = score_s.fillna('0').to_frame().T
    score_df.index = ['score']
    
    # シュート
    shot_s = df_tmp[df_tmp.eventName=='Shot'].groupby('teamId').size()

    penalty_s = df_tmp[df_tmp.subEventName=='Penalty'].groupby('teamId').size()
    for i in penalty_s.index:
        shot_s[i] += penalty_s[i]

    shot_df = shot_s.fillna('0').to_frame().T
    shot_df.index = ['number of shots']
    
    # パス本数
    pass_s = df_tmp[df_tmp.eventName=='Pass'].groupby('teamId').size()

    pass_df = pass_s.fillna('0').to_frame().T
    pass_df.index = ['number of passes']

    # パス成功率
    r_pass_s = df_tmp[df_tmp.eventName=='Pass'].groupby('teamId').tags.agg(lambda xs: np.sum([[1801 in [tag_dict['id'] for tag_dict in tags]] for tags in xs])) 

    r_pass_df = (r_pass_s.to_frame().T / pass_df.values * 100).astype(int)
    r_pass_df = r_pass_df.applymap(lambda x: f'{x: .2f}%')

    r_pass_df.index = ['pass success rate']
    
    # クリア数
    clear_s = df_tmp[df_tmp.subEventName=='Clearance'].groupby('teamId').size()

    clear_df = clear_s.fillna('0').to_frame().T
    clear_df.index = ['number of Clear']

    # ファール
    foul_s = df_tmp[df_tmp.subEventName=='Foul'].groupby('teamId').size()

    foul_df = foul_s.fillna('0').to_frame().T
    foul_df.index = ['number of Foul']
    
    # デュエル勝率
    r_duel_s = df_tmp[df_tmp.eventName=='Duel'].groupby('teamId').tags.agg(lambda xs:np.sum([np.sum([1801 in [tag_dict['id']] for tag_dict in tags]) for tags in xs])/len(xs))

    r_duel_df = (r_duel_s.to_frame().T * 100)
    r_duel_df = r_duel_df.applymap(lambda x: f'{x: .2f}%')

    r_duel_df.index = ['duel win rate']
    
    match_summary_df = pd.concat([score_df, shot_df, pass_df, r_pass_df, clear_df, foul_df, r_duel_df])
    
    match_summary_df.columns = [teams_df[teams_df.wyId==teamId].name.values[0] for teamId in match_summary_df.columns.tolist()]
    
    return match_summary_df

In [106]:
create_match_summary_df(df_tmp, teams_df)

,Germany,France
score,0,2
number of shots,15,12
number of passes,644,270
pass success rate,87.00%,81.00%
number of Clear,5,24
number of Foul,7,12
duel win rate,61.03%,61.54%


# eventごとに

In [10]:
for eventName in ['Pass', 'Duel']:
    
    df_tmp['teamName'] = df_tmp.teamId.apply(lambda x: teams_df[teams_df.wyId==x].name.values[0])
    summary = df_tmp[df_tmp.eventName==eventName].groupby(['teamName', 'subEventName']).size().to_frame()
    summary.columns = ['Number of actions']

    print(eventName)
    display(summary.reset_index(0).pivot_table(values=['Number of actions'], index=['subEventName'], columns=['teamName']).style.bar(vmin=summary.min(), vmax=summary.max()))
    

Pass


/Users/yamamuros83/.pyenv/versions/anaconda3-5.3.1/envs/sample_env/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Duel
